# CONNECTION TO MICROSOFT SQL SERVER DATABASE

In [8]:
#!pip install pyodbc

#!pip install sqlalchemy



In [30]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine

# Step 1: Connect using Windows Authentication
server = r'HP\SQLEXPRESS'
connection_string = f'''
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
Trusted_Connection=yes;
'''

try:
    # Initial connection
    connection = pyodbc.connect(connection_string, autocommit=True)
    cursor = connection.cursor()

    # Step 2: Show all non-system databases
    cursor.execute("SELECT name FROM sys.databases WHERE database_id > 4")
    databases = [row[0] for row in cursor.fetchall()]
    print("Databases:")
    for i, db in enumerate(databases, 1):
        print(f"{i}. {db}")
    
    db_choice = int(input("\n Choose a database by number: ")) - 1
    selected_db = databases[db_choice]
    print(f"\nUsing database: {selected_db}")

    # Reconnect to selected DB
    connection = pyodbc.connect(f'''
        DRIVER={{ODBC Driver 17 for SQL Server}};
        SERVER={server};
        DATABASE={selected_db};
        Trusted_Connection=yes;
    ''')
    cursor = connection.cursor()

    # Step 3: List tables
    cursor.execute("""
        SELECT TABLE_NAME 
        FROM INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_TYPE='BASE TABLE'
    """)
    tables = [row[0] for row in cursor.fetchall()]
    print("\nTables:")
    for i, tbl in enumerate(tables, 1):
        print(f"{i}. {tbl}")

    tbl_choice = int(input("\n Choose a table by number: ")) - 1
    selected_table = tables[tbl_choice]
    print(f"\nViewing table: {selected_table}")

    # Step 4: Load table data into a Pandas DataFrame
    conn_str = (
        f"mssql+pyodbc://{server}/{selected_db}"
        "?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
    )
    engine = create_engine(conn_str)

    query = f"SELECT * FROM [{selected_table}]"  
    df = pd.read_sql(query, engine)

    print(f"\nSpreadsheet-like View of '{selected_table}':")
    print(df.head(20))  # Show top 20 rows like a preview

except Exception as e:
    print("\nSOMETHING WENT WRONG:")
    print(e)


Databases:
1. SalesDatabase
2. Heart
3. PACKT_ONLINE_SHOP
4. LandonHotel
5. TimeSeriesData
6. Abukar
7. db_sql_tutorial
8. olist_DB
9. MyDatabase
10. SalesDB
11. AdventureWorksDW2022



 Choose a database by number:  11



Using database: AdventureWorksDW2022

Tables:
1. DatabaseLog
2. AdventureWorksDWBuildVersion
3. DimAccount
4. DimCurrency
5. DimCustomer
6. DimDate
7. DimDepartmentGroup
8. DimEmployee
9. DimGeography
10. DimOrganization
11. DimProduct
12. DimProductCategory
13. DimProductSubcategory
14. DimPromotion
15. DimReseller
16. DimSalesReason
17. DimSalesTerritory
18. DimScenario
19. FactAdditionalInternationalProductDescription
20. FactCallCenter
21. FactCurrencyRate
22. FactFinance
23. FactInternetSales
24. FactInternetSalesReason
25. FactProductInventory
26. FactResellerSales
27. FactSalesQuota
28. FactSurveyResponse
29. NewFactCurrencyRate
30. ProspectiveBuyer
31. sysdiagrams



 Choose a table by number:  30



Viewing table: ProspectiveBuyer

Spreadsheet-like View of 'ProspectiveBuyer':
    ProspectiveBuyerKey ProspectAlternateKey  FirstName MiddleName   LastName  BirthDate MaritalStatus Gender                      EmailAddress  YearlyIncome  TotalChildren  NumberChildrenAtHome   Education      Occupation HouseOwnerFlag  NumberCarsOwned                 AddressLine1 AddressLine2            City StateProvinceCode PostalCode                Phone Salutation  Unknown
0                     1          21596444800       Adam       None  Alexander 1945-12-30             M      M  aalexander@lucernepublishing.com       40000.0              3                     0  Partial Co    Professional              1                2                  566 S. Main         None      Cedar City                UT      84720         516-555-0187        Mr.        0
1                     2                 3003   Adrienne       None     Alonso 1950-03-06             M      F        aalonso@alpineskihouse.com       80000

In [31]:
from IPython.display import display
import pandas as pd
from sqlalchemy import create_engine

# Setup (reusing previous variables)
conn_str = (
    f"mssql+pyodbc://{server}/{selected_db}"
    "?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
)
engine = create_engine(conn_str)

# Load table
query = f"SELECT * FROM [{selected_table}]"
df = pd.read_sql(query, engine)

# Optional display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Show DataFrame like a spreadsheet
print(f"\n📊 Showing first 20 rows of '{selected_table}':")
display(df.head(20))



📊 Showing first 20 rows of 'ProspectiveBuyer':


,ProspectiveBuyerKey,ProspectAlternateKey,FirstName,MiddleName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,YearlyIncome,TotalChildren,NumberChildrenAtHome,Education,Occupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,City,StateProvinceCode,PostalCode,Phone,Salutation,Unknown
0,1,21596444800,Adam,None,Alexander,1945-12-30,M,M,aalexander@lucernepublishing.com,40000.0,3,0,Partial Co,Professional,1,2,566 S. Main,None,Cedar City,UT,84720,516-555-0187,Mr.,0
1,2,3003,Adrienne,None,Alonso,1950-03-06,M,F,aalonso@alpineskihouse.com,80000.0,4,0,Bachelors,Management,1,2,7264 St. Peter Court,None,Colma,CA,94014,607-555-0119,Ms.,4
2,3,1077,Alfredo,B,Alvarez,1964-03-30,S,M,aalvarez@fineartschool.net,130000.0,3,4,Partial Co,Professional,1,4,8850 Via Del Sol,None,Lynnwood,WA,98036,1 (11) 500 555-0143,Mr.,1
3,4,4779,Arthur,A,Arun,1985-03-22,S,M,aarun@adventure-works.com,50000.0,0,0,Partial Co,Skilled Manual,1,2,7515 Royal Oak Rd.,None,Chula Vista,CA,91910,403-555-0186,Mr.,4
4,5,38032399400,Andrea,M,Bailey,1965-04-26,M,F,abailey@lucernepublishing.com,10000.0,2,0,Partial Co,Manual,1,0,2500 North Stemmons Freeway,None,Dallas,TX,75201,1 (11) 500 555-0113,Ms.,4
5,6,67701756358,Angel,C,Bell,1983-10-12,S,M,abell@thephone-company.com,50000.0,0,0,High Schoo,Professional,0,2,840 Charlotte Ave.,None,Puyallup,WA,98371,516-555-0185,Mr.,1
6,7,14496465921,Anna,S,Bennett,1979-09-04,M,F,abennett@alpineskihouse.com,50000.0,1,1,Bachelors,Skilled Manual,1,0,312 Via Del Verdes,None,Colma,CA,94014,433-555-0112,Ms.,4
7,8,96338913440,Alyssa,L,Bennett,1963-01-12,M,F,abennett@cpandl.com,40000.0,3,0,Partial Co,Skilled Manual,1,2,25136 Jefferson Blvd.,None,Culver City,CA,90232,1 (11) 500 555-0160,Ms.,4
8,9,467,Arturo,J,Bhat,1976-03-13,M,M,abhat@adatum.com,90000.0,0,0,Bachelors,Professional,1,0,7040 Isabel Dr.,None,Beverly Hills,CA,90210,1 (11) 500 555-0199,Mr.,4
9,10,53313373327,Abigail,None,Brown,1955-09-02,M,F,abrown@treyresearch.net,40000.0,5,0,High Schoo,Professional,0,3,4710 Northridge Drive,None,Port Orchard,WA,98366,155-555-0188,Ms.,1


In [32]:
df.shape

(2059, 24)

In [22]:
df.head()

,FinanceKey,DateKey,OrganizationKey,DepartmentGroupKey,ScenarioKey,AccountKey,Amount,Date
0,1,20101229,3,1,1,60,22080.0,2010-12-29
1,2,20101229,3,1,2,60,20200.0,2010-12-29
2,3,20101229,3,1,2,61,2000.0,2010-12-29
3,4,20101229,3,1,1,61,2208.0,2010-12-29
4,5,20101229,3,1,1,62,1546.0,2010-12-29


In [41]:
df.head()

,OrderID,ProductID,UnitPrice,Quantity,Discount
0,10248,11,14.0,12,0.0
1,10248,42,9.8,10,0.0
2,10248,72,34.8,5,0.0
3,10249,14,18.6,9,0.0
4,10249,51,42.4,40,0.0
